In [1]:
# Import our module containing helper functions
import gravann

# Core imports
import numpy as np
import scipy
import pickle as pk
import os
from collections import deque
import torchquad as tquad
tquad.set_log_level("CRITICAL")
from copy import deepcopy

# pytorch
from torch import nn
import torch

# plotting stuff
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

# If possible enable CUDA
gravann.enableCUDA()
gravann.fixRandomSeeds()
device = os.environ["TORCH_DEVICE"]
print("Will use device ",device)

C:\ProgramData\Miniconda3\envs\geodesynet\lib\site-packages\pyvista\themes.py:123: PyvistaDeprecationWarning: use "dark" instead of "night" theme
  warnings.warn('use "dark" instead of "night" theme', PyvistaDeprecationWarning)
12:37:37|TQ-INFO| Initializing torchquad.


Available devices  1
__pyTorch VERSION: 1.10.0
__CUDNN VERSION: 7605
__Number CUDA Devices: 1
Active CUDA Device: GPU 0
Setting default tensor type to Float32
Will use device  cuda:0


In [2]:
# Load the models to compare

In [37]:
name = "planetesimal"

In [41]:
# We load the masconCUBE
with open("models/"+name+"_masconCUBE.pk", "rb") as file:
    mascon_cube_points, mascon_cube_masses, mascon_cube_name = pk.load(file)
# We load the ground truth
with open("mascons/Hollow2.pk", "rb") as file:
    mascon_points, mascon_masses, mascon_name = pk.load(file)
mascon_points = torch.tensor(mascon_points)
mascon_masses = torch.tensor(mascon_masses)
mascon_cube_points = torch.tensor(mascon_cube_points)
mascon_cube_masses = torch.tensor(mascon_cube_masses)

# We load the gravityNet
encoding = gravann.direct_encoding()
model = gravann.init_network(encoding, n_neurons=100, model_type="siren", activation = gravann.AbsLayer())
model.load_state_dict(torch.load("models/"+name+".mdl"))

<All keys matched successfully>

In [42]:
#results_mascon = gravann.validation_mascon(mascon_cube_points, mascon_cube_masses,mascon_points,mascon_masses, asteroid_pk_path="3dmeshes/Eros.pk", N=10000, batch_size=100, progressbar=True)

In [44]:
results_geodesyNet = gravann.validation(model, encoding, mascon_points, mascon_masses, use_acc=True, asteroid_pk_path="3dmeshes/Hollow2.pk", N=10000, N_integration=300000, batch_size=100, progressbar=True)

Computing validation...:  57%|████████████████████████████▍                     | 39300/69204 [00:41<00:31, 952.74it/s]

Discarding 1829 of 12288 points in altitude sampler which did not meet requested altitude.


Computing validation...:  71%|███████████████████████████████████▋              | 49400/69204 [01:16<00:49, 397.00it/s]

Discarding 5600 of 12288 points in altitude sampler which did not meet requested altitude.


Computing validation...:  86%|██████████████████████████████████████████▉       | 59400/69204 [01:49<00:24, 394.36it/s]

Discarding 9745 of 12288 points in altitude sampler which did not meet requested altitude.


Computing validation...: 69400it [02:19, 496.70it/s]                                                                   


In [45]:
results_geodesyNet.to_csv("models/"+name+"_validation_results.csv")

In [46]:
results_geodesyNet

,Altitude,Normalized L1 Loss,Normalized Relative Component Loss,RMSE,relRMSE
0,Low Altitude,0.754842,1.540118,2.293016,1.549731
1,High Altitude,0.048585,0.123796,0.178742,0.147401
2,Altitude_0,0.073792,0.151197,0.376697,0.256801
3,Altitude_1,0.059097,0.136295,0.214249,0.164891
4,Altitude_2,0.032482,0.103354,0.108422,0.115376
